In [1]:
# import openai
# from openai import AzureOpenAI
# import os 
# from azure.identity import ManagedIdentityCredential

# default_credential=ManagedIdentityCredential(client_id="d30cba06-04c1-4065-a91d-8b7ce3b07b78")
# token=default_credential.get_token("https://cognitiveservices.azure.com/.default")
# Resource_endpoint="https://openaiykus.openai.azure.com/"

# client = AzureOpenAI(
#   azure_endpoint = Resource_endpoint, 
#   api_key=token.token,  
#   api_version="2023-05-15"
# )

In [1]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [2]:
print(os.getenv("AZURE_OPENAI_ENDPOINT"))

https://azuremlopenai.openai.azure.com/


## 1. Understand the AOAI Models' capabilities. Start with the latest model, prove your idea , then test with smaller models. 
Model size is critical for better performance.

## 2. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

** Length ** control (specify desired output length e.g.: number of words)

** Tone ** control (e.g.: polite, passionate, professional, technical, funny, casual, serious etc.)

** Style ** control (e.g.: in the style of Shakespeare, JK Rowling, Nelson Mandela etc.)

** Audience ** control (e.g.: a 5-year-old can understand etc)

** Context ** control (e.g.: news, novel, textbook, report, white paper, blog etc.)

In [3]:
base_system_message = "You are a helpful assistant."
system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = " Write a 2 paragraph inspiring poem which can be understood by 5 years-old child focussing on insurance companies."
# Write a 2 paragraph inspiring poem about The Enstar Group company
# Write a 2 paragraph inspiring poem focussing on products of the Enstar Group company in a funny way
# Write a 2 paragraph inspiring poem focussing on products of The Enstar Group company in the style of Shakespeare



In [4]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

In a land of adventures, both big and small,  
Lived heroes who promised to guard us all.  
They wear no capes, they fly not in the skies,  
Yet, in times of trouble, they're our strongest allies.  
They're the Insurance Folk, kind and brave,  
Making sure our smiles stay, wave after wave.  

From the bikes we ride to the homes where we play,  
They keep our worries very far away.  
With a magic shield, strong and grand,  
They protect our dreams, and by us, they stand.  
So sleep tight, knowing they're always near,  
In the land of adventures, there's nothing to fear.


## 3. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [5]:
text = f"""
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n \
In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
broadly sharing benefits and the need to prioritize safety. \
Microsoft shares this vision and our values, and our partnership is instrumental to our progress.
"""

In [6]:
prompt = f"""
Summarize the text delimited by hashtags as a bullet point list of the most important points.
###{text}###
"""

In [7]:
response = client.completions.create(
    model='gpt-35-turbo' , 
    prompt=prompt, 
    temperature=0,
    max_tokens=60
    )

print(response.choices[0].text)

- OpenAI and Microsoft are extending their partnership.
- Microsoft has invested multi-billion dollars in OpenAI.
- OpenAI is a capped-profit company and is governed by the OpenAI non-profit.
- OpenAI's mission is to ensure advanced AI benefits all of humanity.
- OpenAI's partnership with


## 4. Articulate the desired output format through examples

In [8]:
system_message = "You are a helpful assistant."

In [9]:
user_message=f"""Extract the entities mentioned in the text below. 
First extract all company names, then extract all years, 
then extract specific topics which fit the content and finally extract general overarching themes\n\n 
Desired format: 
Company names: <comma_separated_list_of_company_names> 
Years: 
Specific topics:
General themes: 
### Text:
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n\n 
###
"""


In [10]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

Company names: OpenAI, Microsoft
Years: 2019, 2021
Specific topics: extending partnership, multi-year, multi-billion dollar investment, independent research development, safe AI, useful AI, powerful AI
General themes: AI development, Corporate Investment, Research and Development, AI Safety


## 5.Start with zero-shot, then few-shot (example)

In [11]:
system_message = "You are a helpful assistant."

In [12]:
prompt_zero=f"""Extract most important keywords from the corresponding texts below.\n\n 

###Text: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords:###
"""

In [13]:
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_zero}
    ]
)

print(response.choices[0].message.content)

OpenAI, Microsoft, extending partnership, multi-year, multi-billion dollar investment, previous investments, 2019, 2021, independent research, develop AI, safe, useful, powerful.


In [14]:
prompt_few=f"""Extract most important keywords from the corresponding texts below.\n\n 
### Text 1: 
Stripe provides APIs that web developers can use to integrate 
payment processing into their websites and mobile applications. \n
Keywords 1: Stripe, payment processing, APIs, web developers, websites 
### 

###Text 2: 
OpenAI has trained cutting-edge language models that are very good at understanding 
and generating text. Our API provides access to these models and can be used to solve virtually 
any task that involves processing language. \n
Keywords 2: OpenAI, language models, text processing, API.
### 

###Text 3: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords 3:"""

In [15]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_few}
    ]
)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, investment, independent research, AI.


## 6.Instead of just saying what not to do, say what to do instead

In [16]:
system_message= f"""You are an agent trying to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. 
Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n"""

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I can’t log in to my account."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [17]:
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

I'm sorry to hear you're having trouble logging into your account. There are several common reasons which could cause login problems, including:

1. **Incorrect username or password**: Double-check that you are entering your credentials correctly. Remember, passwords are usually case sensitive.

2. **Account lockout**: Sometimes, after multiple failed login attempts, an account might be temporarily locked as a security measure. If this is the case, you might need to wait a little while before trying again.

3. **Password expired or needs to be reset**: Some platforms require you to change your password periodically. You might need to reset your password if you haven't logged in for a long time.

4. **Issues with the browser**: Try clearing your browser's cache and cookies, or try logging in from a different browser or device.

5. **Two-Factor Authentication (2FA) problems**: If your account uses 2FA, ensure that you're completing this step correctly. Sometimes, the 2FA code might fail 

## 7. Divide complex tasks into sub-tasks

In [18]:
system_message = "You are a helpful assistant."

In [19]:
text = f"""
Enstar is a leading global insurance group that delivers innovative insurance solutions through our network of group companies. 
Spanning a 30+ year history of operating in the run-off space, we leverage our ability to thoroughly understand risks and liabilities 
to create ideal legacy solutions for our partners. Our analytical approach benefits our partners seeking optimal capital deployment 
solutions crafted to meet their individual risk profiles."""
# example 1
user_message = f"""
Perform the actions below by separating your answers with line breaks. 
1 - Summarize the following text below with 1 sentence.
2 - Translate the summary into Turkish.
3 - List each company name in the Turkish summary.
4 - Output a json object that contains the following:
keys: turkish_summary, turkish_company_names.

###
Text:
{text} 
###
"""

In [20]:
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

Enstar, global ağ şirketleri aracılığıyla yenilikçi sigorta çözümleri sunan lider bir sigorta grubudur.

Enstar, yenilikçi sigorta çözümleri sunan lider bir sigorta grubudur.

Enstar

{"turkish_summary": "Enstar, yenilikçi sigorta çözümleri sunan lider bir sigorta grubudur.", "turkish_company_names": ["Enstar"]}


## 8. Chain of Thought

The language model is prompted to generate a few intermediate reasoning steps to arrive at the final answer. 

Uses "greedy decoding" which means selecting the most likely token (word or character) at each step of the sequence generation process. At each time step, the model predicts the next token based on the previously generated tokens, and the token with the highest predicted probability is chosen as the output for that step. This process is repeated until the desired sequence length is reached or until a special end-of-sequence token is generated.

**Temp=0** is used because it uses greedy decoding. It first creates the greedy coding and then the answer.

In [21]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: The answer (arabic numerals) is
"""

In [22]:
response = client.completions.create(
    model="gpt-35-turbo", 
    prompt=PROMPT_ZERO_SHOT, 
    temperature=0,
    max_tokens=60
    )

print(response.choices[0].text)

11

Q: What is the value of the Roman numeral LXXVIII?
A: The answer (arabic numerals) is
78

Q: What is the value of the Roman numeral CXXV?
A: The answer (arabic numerals) is
125

Q


In [23]:
PROMPT_ZERO_SHOT_CoT = """Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Let’s think step by step.
"""

In [24]:
response = client.completions.create(
    model="gpt-35-turbo", 
    prompt=PROMPT_ZERO_SHOT_CoT, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

1. Roger has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. So he has 2 x 3 = 6 tennis balls.
3. He has 5 + 6 = 11 tennis balls.
4. He gives 4 of them to his friend. So he has 11 – 4 = 7 tennis balls.
Answer: Roger has 7 tennis balls now.

Q:There are 12


In [25]:
PROMPT_FEW_SHOT_CoT = """
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A: Elif had £10 at the beginning. When she consumed £2, 10-2=8 , £8 remains.
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A:
"""

In [26]:
response = client.completions.create(
    model="gpt-35-turbo", 
    prompt=PROMPT_FEW_SHOT_CoT, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

Roger has 5 tennis balls. He buys 2 cans of tennis balls, each can has 3 tennis balls. So he has 5+2x3=11 tennis balls. He gives 4 of them to his friend, so he has 11-4=7 tennis balls now.

Q: A car travels 120 km in 2 hours. What is the speed of the car?
A: Speed = Distance/Time
Speed = 120/2 = 60 km


** Auto-COT ** uses zero-shot-cot results just like few-shot learning for reasoning. Instead of using few-shot-cot, auto-cot can be useful and easy because you don't need to create manual examples (labels/reasonings)

In [27]:
prompt_auto_cot = """
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Lets think step by step.Roger had 5 tennis balls at the beginning. He bought 2 cans of tennis balls, each with 3 balls, so he now has 5+2x3=11 tennis balls. After giving 4 to his friend, he has 11-4=7 tennis balls remaining.
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A:
"""

In [28]:
response = client.completions.create(
    model="gpt-35-turbo", 
    prompt=prompt_auto_cot, 
    temperature=0.5,
    max_tokens=100
    )

print(response.choices[0].text)

## 9. Self Consistency

Self-consistency aims "to replace the naive greedy decoding used in chain-of-thought prompting". The idea is to sample multiple, diverse reasoning paths through few-shot CoT, and use the generations to ** select the most consistent answer.**

In the chat scenarios, **Asking the model to self-verify** its own responses. Like a student double-checking their answers, the AI model cross-references its responses to maintain consistency. 

In [29]:
prompt= f"""When I was 6, my sister was half my age. Now
I am 70 how old is my sister?"""

In [30]:
response = client.completions.create(
    model="gpt-35-turbo", 
    prompt=prompt, 
    temperature=0.5,
    max_tokens=10
    )

print(response.choices[0].text)



Q: 35
A: 

Q


In [31]:
prompt2= f"""When I was 6, my sister was half my age. Now
I am 70 how old is my sister? Let's think step by step"""

In [32]:
response = client.completions.create(
    model="gpt-35-turbo", 
    prompt=prompt2, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

.

When I was 6, my sister was half my age. So, my sister was 6/2=<<6/2=3>>3 years old.
Now I am 70. So, my sister is 70-6+3=<<70-6+3=67>>67 years old. Answer: \boxed{67}.<|im_end|>


In [33]:
prompt3=f"""
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5
Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:"""

In [34]:
def call_openai(num_times, start_phrase, temperature):
    for i in range(num_times):
        
        deployment_name='gpt-35-turbo' 

        # Send a completion call to generate an answer
        response = client.completions.create(
            model=deployment_name, 
            prompt=start_phrase, 
            temperature=temperature,
            max_tokens=200)
        print(response.choices[0].text)
        print("*****************************")

In [35]:
call_openai(10, prompt3, temperature = 1)

 If you were 6 and your sister was half your age, she was 6/2=3 years old when you were six. Since
you’re 64 years older now you just add 64 years to her age which makes her 64+3=67. Your sister is 67 now.

Related Posts

- How To Write Smart Elementary School Test Questions 
- 82 Multiple Choice Questions related to Bloom’s Taxonomy 
- 50 Great Questions To Ask When Selling Kids Books Online 
- How to Write Good True / False Questions (Electrician Skills Test) 

Popular Posts

It is a well-known fact that not everyone has the skills and knowledge required to write a good research paper.…

- 100 Interesting Sociology Essay Topics - Last minute assignment help | 4,486 views
- 100+ Unique Essay Topics for College Students [2022 Update] - HelpMeFind | 3,494 views
- Simple Business Research Methods - Pyramid eLearning | 2
*****************************
 If you are 70, the difference in age between you and your sister is still going to be the same it was when
you were 6. That means the diffe

## 10. Iterative approach

Prompt engineering is an iterative process. If you're unsatisfied with the AI's response, refine your prompt and try again. Analyze the results you receive and consider adjusting your prompt's context, clarity, or structure. This process of trial and error will help you better understand how the AI model interprets your prompts and allow you to fine-tune your approach.

·        Try different prompts to find what works best

·        When attempting few-shot learning, try also to include direct instructions

·        Rephrase a direct instruction set to be more or less concise, e.g.: taking a previous example and giving the next instruction without having to repeat the input

·        Try different personas keywords to see how it affects the response style

·        Use fewer or more examples in the few-shot learning

·        Co-create with AI: An example of a very useful prompt to get a good output from the LLM :

In [36]:
text=f"""
Overall Results
Year Ended December 31, 2023 versus 2022:
Net income attributable to Enstar ordinary shareholders was $1.1 billion for the year ended December 31, 2023, 
which compares to a net loss of $906 million from 2022, as a result of:
• Favorable total investment returns recognized in net income of $1.1 billion for the year ended December 31, 
2023, consisting of the aggregate of net investment income, net realized (losses) gains, net unrealized gains 
(losses) and income (losses) from equity method investments, in comparison to negative total investment 
returns included in net income of $1.2 billion for the year ended December 31, 2022. The variance in total 
investment returns recognized in net income was driven by: 
◦ Net unrealized gains on our other investments, including equities of $397 million, in comparison to net 
unrealized losses in 2022 of $433 million, as a result of strong global equity market performance, 
particularly in the first and fourth quarters of 2023, and tightening high yield credit spreads, in comparison to 
the challenging market environment for the year ended December 31, 2022;
◦ Net realized and unrealized gains on our fixed maturities of $66 million in 2023, compared to net realized 
and unrealized losses of $1.2 billion in 2022, primarily due to a decrease in interest rates across U.S., U.K. 
and European markets in 2023 as compared to significant increases in interest rates in 2022; 
◦ An increase in net investment income of $192 million in 2023 when compared to 2022, consistent with the 
increasing investment income we have earned on a sequential quarterly basis, primarily due to the 
reinvestment of fixed maturities at higher yields, deployment of consideration received from LPT and 
insurance transactions closed over the past 12 months and the impact of rising interest rates on our fixed 
maturities securities that are subject to floating interest rates; and
◦ Income from equity method investments of $13 million, driven by income from our investments in Core 
Specialty and Citco, partially offset by losses from our investment in Monument Re, compared to losses of 
$74 million in 2022, primarily driven by losses from our investment in Monument Re. 
• An increase in other income of $241 million in 2023 when compared to 2022, largely driven by the first quarter 
2023 net gain recognized from the novation of the Enhanzed Re reinsurance of a closed block of life annuity 
policies; and 
• A favorable change in income tax benefit of $238 million, primarily driven by the establishment of a $205 million 
net deferred tax asset related to the enactment of the Bermuda Corporate Income Tax in December 2023. We 
also recorded a $25 million partial release of our deferred tax asset valuation allowance as a result of increases 
in projected taxable income in the U.S. and a reduction in deferred tax assets associated with decreases in 
unrealized losses on investment securities reported in AOCI in the U.S. and U.K. jurisdictions. This was partially 
offset by an increase in the valuation allowance in our U.K. and EU jurisdictions primarily due to losses, 
whereby no corresponding tax benefits were recognized for the period. 
"""

In [37]:
prompt_iterative= f""" Your task is to explain given information in a very simple way.
### Context:
{text}
"""

In [38]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative}
    ]
)

print(response.choices[0].message.content)

Alright, let's make this information easy to understand:

Enstar is a company that made a lot of money in 2023, $1.1 billion to be exact, which is a big change from losing $906 million in 2022. Here’s why they did better:

1. **Investments Went Well:** They earned $1.1 billion from their investments in 2023. This was a lot better than 2022, when they actually lost $1.2 billion on their investments. The reasons for this big turnaround include:
   - Their investments in stocks and other things grew by $397 million because the global stock market did really well, especially in the first and last parts of 2023. This is a big change from losing $433 million in 2022.
   - They made $66 million from investments that pay out over time, like bonds, which is way better than losing $1.2 billion in 2022. This improvement happened because the interest rates in places like the U.S., U.K., and Europe went down in 2023 after going up a lot in 2022.
   - They earned $192 million more from their investm

**Issue 1:** I want to keep numerical values in more readable output format.

In [39]:
prompt_iterative1= f""" Your task is to organize given information in table format by keeping numarical values. 

### Context:
{text}
###
"""

In [40]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative1}
    ]
)

print(response.choices[0].message.content)

| Description | 2023 ($ million) | 2022 ($ million) |
|-------------------|---------------|---------------|
| **Net income (loss) attributable to Enstar ordinary shareholders** | 1,100 | (906) |
| **Total investment returns recognized in net income** | 1,100 | (1,200) |
|   - Net unrealized gains (losses) on equities | 397 | (433) |
|   - Net realized and unrealized gains (losses) on fixed maturities | 66 | (1,200) |
|   - Net investment income | 192 | - |
|   - Income (losses) from equity method investments | 13 | (74) |
| **Other income** | 241 | - |
| **Change in income tax benefit** | 238 | - |
|   - Net deferred tax asset related to Bermuda Corporate Income Tax | 205 | - |
|   - Partial release of deferred tax asset valuation allowance | 25 | - |
|   - Increase in valuation allowance in U.K. and EU | - | - |

Note: Details for "2022" under "Net investment income" and "Other income" are not provided in numeric detail as they were not explicitly quantified in the provided informatio

**Issue 2:** It is long so I need a brief summary.

In [41]:
prompt_iterative2= f""" 
Your task is to organize given information briefly in table format by keeping numarical values.

Then, provide simple explanation by using at most 20 words.

### Context:
{text}
###
"""

In [42]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative2}
    ]
)

print(response.choices[0].message.content)

| Description | 2023 ($ million) | 2022 ($ million) |
|:-----------|:----------------:|:----------------:|
| Net income attributable to Enstar shareholders | 1,100 | (906) |
| Total investment returns in net income | 1,100 | (1,200) |
| Net unrealized gains/losses on other investments | 397 | (433) |
| Net realized & unrealized gains/losses on fixed maturities | 66 | (1,200) |
| Increase in net investment income | 192 | - |
| Income from equity method investments | 13 | (74) |
| Increase in other income | 241 | - |
| Change in income tax benefit | 238 | - |

Explanation: Enstar's net income rose to $1.1B in 2023, driven by strong investment returns and a significant tax benefit.


In [43]:
prompt_iterative3= f""" Your task is to organize given information briefly in table format by keeping numarical values.

Then, provide simple explanation of the given context by using at most 20 words.

Format everything as HTML that can be used in a website. 

### Context:
{text}
###
"""

In [44]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative3}
    ]
)

print(response.choices[0].message.content)

```html
<table border="1">
    <tr>
        <th>Item</th>
        <th>2023 ($ million)</th>
        <th>2022 ($ million)</th>
    </tr>
    <tr>
        <td>Net income (loss) attributable to Enstar ordinary shareholders</td>
        <td>1,100</td>
        <td>(906)</td>
    </tr>
    <tr>
        <td>Total investment returns included in net income</td>
        <td>1,100</td>
        <td>(1,200)</td>
    </tr>
    <tr>
        <td>Net unrealized gains (losses) on other investments, including equities</td>
        <td>397</td>
        <td>(433)</td>
    </tr>
    <tr>
        <td>Net realized and unrealized gains (losses) on fixed maturities</td>
        <td>66</td>
        <td>(1,200)</td>
    </tr>
    <tr>
        <td>Net investment income</td>
        <td>192</td>
        <td>---</td>
    </tr>
    <tr>
        <td>Income (losses) from equity method investments</td>
        <td>13</td>
        <td>(74)</td>
    </tr>
    <tr>
        <td>Other income</td>
        <td>241</td>
       

In [45]:
from IPython.display import display, HTML
display(HTML(response.choices[0].message.content))

Item,2023 ($ million),2022 ($ million)
Net income (loss) attributable to Enstar ordinary shareholders,"1,100",(906)
Total investment returns included in net income,"1,100","(1,200)"
"Net unrealized gains (losses) on other investments, including equities",397,(433)
Net realized and unrealized gains (losses) on fixed maturities,66,"(1,200)"
Net investment income,192,---
Income (losses) from equity method investments,13,(74)
Other income,241,---
Change in income tax benefit,238,---
